<a href="https://colab.research.google.com/github/jhtwiz/AI-1-jhtwiz/blob/main/4%EC%A3%BC%EC%B0%A8_%EA%B8%B0%EB%B3%B8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HuggingFace로 뉴스 기사 분류하기

In [ ]:
!pip install transformers datasets evaluate accelerate scikit-learn

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 11.0/11.0 MB 85.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ---------------------- ----------------- 24.6/44.5 MB 120.1 MB/s eta 0:00:01
   ---------------------------------- ----- 38.8/44.5 MB 117.5 MB/s eta 0:00:01
   ----------------------------------- ---- 39.8/44.5 MB 65.0 MB/s eta 0:00:01
   ------------------------------------- -- 41.9/44.5 MB 58.1 MB/s eta 0:00:01
   -------------------------------------- - 43.0/44.5 MB 42.1 MB/s eta 0:00:01
   ---------------------------------------  44.3/44.5 MB 38.1 MB/s eta 0:00:01
   ---------------------------------------- 44.5/44.5 MB 34.6 MB/s eta 0:00:00


In [ ]:
import random
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

C:\Users\Dryhand\anaconda3\envs\mlTest\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
db = load_dataset("fancyzhx/ag_news")
db

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [ ]:
db['train'][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}

In [ ]:
db['train'].features['label'].names

['World', 'Sports', 'Business', 'Sci/Tech']

In [ ]:
len_classes = len(db['train'].features['label'].names)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def preprocess_function(data):
    return tokenizer(data["text"])

db_tokenized = db.map(preprocess_function, batched=True)

C:\Users\Dryhand\anaconda3\envs\mlTest\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dryhand\.cache\huggingface\hub\models--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\Dryhand\anaconda3\envs\mlTest\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarn

In [ ]:
db_tokenized['train'][0].keys()

dict_keys(['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
db_split = db_tokenized['train'].train_test_split(test_size=0.2)
db_train, db_val = db_split['train'], db_split['test']
db_test = db_tokenized['test']

In [ ]:
len(db_train), len(db_val), len(db_test)

(96000, 24000, 7600)

In [ ]:
from transformers import BertConfig

config = BertConfig()

config.hidden_size = 128  # BERT layer의 기본 hidden dimension
config.intermediate_size = 256  # FFN layer의 중간 hidden dimension
config.num_hidden_layers = 5  # BERT layer의 개수
config.num_attention_heads = 4  # Multi-head attention에서 사용하는 head 개수
config.num_labels = len_classes  # 마지막에 예측해야 하는 분류 문제의 class 개수

model = AutoModelForSequenceClassification.from_config(config)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='hf_transformer',  # 모델, log 등을 저장할 directory
    num_train_epochs=10,  # epoch 수
    per_device_train_batch_size=64,  # training data의 batch size
    per_device_eval_batch_size=64,  # validation data의 batch size
    logging_strategy="epoch",  # Epoch가 끝날 때마다 training loss 등을 log하라는 의미
    do_train=True,  # 학습을 진행하겠다는 의미
    do_eval=True,  # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=1e-3,  # optimizer에 사용할 learning rate
    load_best_model_at_end=True  # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")


def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import EarlyStoppingCallback


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=db_train,
    eval_dataset=db_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.413300,0.287420,0.903750
2,0.213300,0.271459,0.911458
3,0.146600,0.270898,0.917250
4,0.106700,0.331676,0.911375
5,0.074300,0.362094,0.911708
6,0.049500,0.431357,0.906583


TrainOutput(global_step=9000, training_loss=0.16727314164903428, metrics={'train_runtime': 1020.6157, 'train_samples_per_second': 940.609, 'train_steps_per_second': 14.697, 'total_flos': 357627144416256.0, 'train_loss': 0.16727314164903428, 'epoch': 6.0})

In [ ]:
trainer.evaluate(db_test)

{'eval_loss': 0.26234209537506104,
 'eval_accuracy': 0.9219736842105263,
 'eval_runtime': 3.4501,
 'eval_samples_per_second': 2202.846,
 'eval_steps_per_second': 34.492,
 'epoch': 6.0}

In [ ]:
trainer.save_model()

In [ ]:
from transformers import pipeline


classifier = pipeline("sentiment-analysis", model="./hf_transformer/", device='cuda')
test_text = "UK charges 8 in terror plot linked to alert in US LONDON, AUGUST 17: Britain charged eight terror suspects on Tuesday with conspiracy to commit murder and said one had plans that could be used in striking US buildings that were the focus of security scares this month."
print(classifier(test_text))

[{'label': 'LABEL_0', 'score': 0.996440589427948}]
